In [11]:
from pathlib import Path 
import sys, os
path = Path(os.path.dirname(os.path.abspath('')))
if path not in sys.path:
    sys.path.append(os.path.join(path))

from koebe.algorithms.incrementalConvexHull import randomConvexHullE3
from koebe.graphics.spherical2viewer import *
from random import randint

poly = randomConvexHullE3(500) # Generate a random polyhedron

viewer = S2Viewer(600,600) # Create a viewer object
viewer.toggleSphere() # Hide the unit sphere
viewer.addAll(poly.edges + poly.faces) # Add the edges and faces to the viewer

# Style all of the edges black with weight 0.5
for e in poly.edges:
    viewer.setStyle(e, makeStyle(stroke="#000", strokeWeight = 0.5))

# Set each face to a random color:
for i in range(len(poly.faces)):
    h, l = str(randint(0, 360)), str(randint(50, 75))
    viewer.setStyle(poly.faces[i], makeStyle(fill = f"hsl({h}, 100%, {l}%)"))

viewer.show() # Show the viewer

<IPython.core.display.Javascript object>

S2Sketch(height=600, objects='[[{"type": "SegmentE3", "endpoints": [[-0.03893299906420063, -0.9966230013955891…

In [12]:
from koebe.algorithms.poissonDiskSampling import slowAdaptiveDartThrowing, slowAdaptiveDartThrowingWithBoundary

from koebe.geometries.euclidean2 import PointE2
from koebe.geometries.spherical2 import PointS2
from koebe.geometries.euclidean3 import PointE3
from koebe.graphics.euclidean2viewer import UnitScaleE2Sketch, makeStyle
from koebe.graphics.spherical2viewer import S2Viewer

from koebe.algorithms.incrementalConvexHull import incrConvexHull, orientationPointE3

from random import random
import math

# The radius function determines the radius of a dart hitting the point (x, y)
radius_function = (lambda x, y: (1.0 - math.sqrt(y)) * 0.1 + 0.01)

# The adaptive sampling method takes the radius function as input to determine sample radii
samples = slowAdaptiveDartThrowingWithBoundary(radius_function, stop_count = 1000)

# Convert the samples, which are tuples of floats (x, y) to PointE2 objects
samplePoints = [PointE2(2*sample[0] - 1, 2*sample[1] - 1) for sample in samples]

viewer = UnitScaleE2Sketch()
viewer.addAll(samplePoints)
viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(height=500, objects='[[{"type": "PointE2", "point": [1, 0.9327685471727749]}, {"type": "PointE2", "po…

In [13]:
s2points = [PointS2.sgProjectFromPointE2(p) for p in samplePoints]

pts = [(sp.directionE3.vec * (1 + random() / 10000)).toPointE3() for sp in s2points]

mesh = incrConvexHull(pts + [PointE3(0,0,-1)], orientationPointE3)
mesh.outerFace = mesh.verts[-1].remove()


viewer = S2Viewer()
viewer.toggleSphere()
viewer.addAll(pts)
viewer.addAll(mesh.edges)
viewer.addAll(mesh.faces)
viewer.show()

<IPython.core.display.Javascript object>

S2Sketch(height=500, objects='[[{"type": "PointE3", "point": [0.6968665231204373, 0.6500151743443932, -0.30315…

In [14]:
mesh2 = mesh.duplicate(
    vdata_transform = (lambda v : PointS2(v.x, v.y, v.z).sgProjectToPointE2())
)
viewer = UnitScaleE2Sketch()
viewer.addAll(mesh2.verts)
viewer.addAll(mesh2.edges)
viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(height=500, objects='[[{"type": "PointE2", "point": [0.6354297345975858, 0.9999999999999999]}, {"type…